# Final NeuroAI Project
## Group 1: Femke, Tikva and Gabriela

In [ ]:
# Importing everything important. 
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc as sp
import matplotlib.image as mpimg
import os
from glob import glob
from PIL import Image
import random
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

In [ ]:
# Setting the standard seed for reproducable results.
random.seed(10) 

## Helper Functions


In [ ]:
# This loads the images from the data set in black and white. While keeping the ratio of cats and dogs equal 
# and keeping track of the picture labels. 
def load_balanced_images(folder, image_size=(32, 32), n_per_class=None):
    """Load the same number of grayscale images per class (e.g. cats/dogs)."""
    classes = sorted(os.listdir(folder))  # ['cats', 'dogs']
    all_images, all_labels = [], []
    
    for label in classes:
        class_folder = os.path.join(folder, label)
        paths = []
        for ext in ('*.png', '*.jpg', '*.jpeg'):
            paths.extend(glob(os.path.join(class_folder, ext)))
        random.shuffle(paths)
        
        if n_per_class:
            paths = paths[:n_per_class]  # limit per class
        
        # Going through each image and giving it an index.
        for idx, path in enumerate(paths):
            img = Image.open(path).convert('L').resize(image_size)
            img = np.array(img, dtype=np.float32) / 255.0
            all_images.append(img)
            all_labels.append(f"{label}_{idx+1}")  # UNIQUE label, e.g. cat_1, dog_3
    
    # Mix the dataset
    combined = list(zip(all_images, all_labels))
    random.shuffle(combined)
    all_images, all_labels = zip(*combined)
    
    print(f"Loaded {len(all_images)} images from {folder} "
          f"({len(classes)} classes, {n_per_class} per class)")
    
    return list(all_images), list(all_labels)


# This code generates the noisy picture.
def imageGenerator(imageVector, binary=None):
    imageVector = imageVector.astype(float)
    imageVector /= np.max(imageVector)
    # If binary is set to True, then the image gets binarised. Otherwise (else) it is not, thus staying in grayscale.
    if binary: 
        cleanImage = np.where(imageVector > 0.5, 1, -1)
        noisyImage = cleanImage + np.random.normal(0, 0.8, cleanImage.shape)
        noisyImage = np.where(noisyImage >= 0, 1, -1)
    else: 
        # Continuous case
        cleanImage = 2 * (imageVector / np.max(imageVector)) - 1  # in [-1, 1]
        noisyImage = cleanImage + np.random.normal(0, 0.15, cleanImage.shape) # Making it less noisier when grayscale, otherwise totally random picture with white/black dots.
        noisyImage = np.clip(noisyImage, -1, 1)  # keep within range, no binarization
    return cleanImage, noisyImage


# Here happens they hebbian learning. 
def hebbian_trainer(pattern, old_weights=None):
    """
    Simple Hebbian learning rule for Hopfield network.
    
    pattern : 1D numpy array of ±1 values
    weights : existing weight matrix or None (for first pattern)
    """
    # Flatten the image to get the pattern.
    pattern = pattern.flatten().astype(np.float32)
    N = len(pattern)

    # Inititalizing new_weights with all zeros.
    new_weights = np.zeros((N, N))

    # --- Hebbian learning ---
    for i in range(N):
        for j in range(N):
            if i != j:  # no self-connection
                new_weights[i, j] = pattern[i] * pattern[j]

    # Normalize by vector length to avoid huge activations, it prevents saturation.
    new_weights /= N

    # If its the first iteration the new weigths are the first weights otherwise we update the weight matrix.
    if old_weights is None:
        return new_weights
    else:
        return old_weights + new_weights

# This returns the pattern.
def prediction(corruptedVec, coefMat, binary):
    """Hopfield recall: one synchronous update."""
    corruptedVec = corruptedVec.flatten()
    
    # If binary, we want to recall a binary pattern. Otherwise grayscale difference in sign and tanh function.
    if binary:
        predictVec = np.sign(np.dot(coefMat, corruptedVec))
        predictVec[predictVec == 0] = 1  # handle zeros
    else: 
        predictVec = np.tanh(np.dot(coefMat, corruptedVec))

    side = int(np.sqrt(len(predictVec)))
    return predictVec.reshape((side, side))


## Visualization helper functions

In [ ]:
# This plots the four pictures side by side. (The original, the cleaned, the noisy and the recalled.)
def plot_image_reconstruction(selected_images, selected_labels, coefMatrix, imageGenerator, prediction, save_path=None, binary=None):

    n_images = len(selected_images)
    plt.figure(figsize=(15, 5 * n_images))

    for i, img in enumerate(selected_images):
        clean, noisyVec = imageGenerator(img, binary)
        predictedVec = prediction(noisyVec, coefMatrix, binary)

        # Original 
        plt.subplot(n_images, 4, 4 * i + 1)
        plt.imshow(img, cmap='gray')
        plt.title(f'Original ({selected_labels[i]})')
        plt.axis('off')

        # Cleaned 
        plt.subplot(n_images, 4, 4 * i + 2)
        plt.imshow(clean, cmap='gray')
        plt.title('Cleaned')
        plt.axis('off')

        # Noisy
        plt.subplot(n_images, 4, 4 * i + 3)
        plt.imshow(noisyVec, cmap='gray')
        plt.title('Noisy')
        plt.axis('off')

        # Recalled
        plt.subplot(n_images, 4, 4 * i + 4)
        if binary: 
            plt.imshow(predictedVec, cmap='gray')
        else: 
            plt.imshow((predictedVec + 1) / 2, cmap='gray')
        plt.title('Recalled')
        plt.axis('off')


    plt.tight_layout()

    # Saving the image
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')

    plt.show()
    plt.close()


# This creates the confusion matrix.
def plot_internal_recall_confusion(
        learning_rule,
        image_size,
        stored_patterns,
        stored_labels,
        coefMatrix,
        imageGenerator,
        prediction,
        binary,
        n_samples=10,
        color = None,
        save_path=None
    ):
    idxs = np.random.choice(len(stored_patterns), n_samples, replace=False)
    y_true, y_pred = [], []

    for i in idxs:
        clean = stored_patterns[i]
        noisy_img = imageGenerator(clean, binary)[1]
        recalled = prediction(noisy_img, coefMatrix, binary)

        # Restrict to sampled subset
        similarities = [np.mean(recalled == stored_patterns[j]) for j in idxs]
        best_match_local = np.argmax(similarities)
        best_match_idx = idxs[best_match_local]

        y_true.append(i)
        y_pred.append(best_match_idx)

    # Compute accuracy
    accuracy = np.mean(np.array(y_true) == np.array(y_pred))
    print(f"Internal recall accuracy ({n_samples} samples): {accuracy:.3f}")

    # --- Label names for the selected subset ---
    labels = [stored_labels[i] for i in idxs]

    # --- Confusion matrix ---
    cm = confusion_matrix(y_true, y_pred, labels=idxs)

    plt.figure(figsize=(8, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap=color, values_format='d', colorbar=False)

    plt.title(f"Model {learning_rule}, binary:{binary}, neurons:{image_size},(accuracy={accuracy:.2f}), patterns={len(stored_patterns)}", fontsize=14)

    # Rotate and clean up label text
    plt.xticks(rotation=45, ha='right', fontsize=8)
    plt.yticks(rotation=0, fontsize=8)
    plt.xlabel("Predicted pattern", fontsize=10)
    plt.ylabel("True pattern", fontsize=10)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.show()

    return accuracy, cm


## Main Experiment

In [106]:
def run_hopfield_experiment(train_dir="data/train", train_counts=None, 
                            learning_rule="hebbian", hebbian_trainer=None, color = "Blues",
                            imageGenerator=None, prediction=None, plot_image_reconstruction=None, image_size=(32, 32), binary=None):
   
    if train_counts is None:
        train_counts = list(range(1, 13))

    # --- Load images ---
    train_images, train_labels = load_balanced_images("data/train", image_size=image_size, n_per_class=75)
   
    results = []
    conf_matrices = []
    train_labels_list = []

    for n_train in train_counts:
        print(f"\n=== Training with {n_train} patterns ===")
        selected_indices = random.sample(range(len(train_images)), n_train)
        selected_images = [train_images[i] for i in selected_indices]
        selected_labels = [train_labels[i] for i in selected_indices]

        coefMatrix = 0
        stored_patterns = []

        # --- Learning phase ---
        for i, img in enumerate(selected_images):
            clean, noisyVec = imageGenerator(img, binary)

            if learning_rule.lower() == "hebbian":
                coefMatrix = hebbian_trainer(clean, 0 if i == 0 else coefMatrix)
            else:
                raise ValueError("Invalid learning_rule. Choose 'hebbian'.")

            stored_patterns.append(clean)

        # --- Visualization ---
        plot_image_reconstruction(
            selected_images=selected_images,
            selected_labels=selected_labels,
            coefMatrix=coefMatrix,
            imageGenerator=imageGenerator,
            prediction=prediction,
            save_path=f"hopfield_stage_{learning_rule}_{n_train}_binary{binary}_neurons{image_size}.png",
            binary = binary
        )

        acc, cm = plot_internal_recall_confusion(
            image_size=image_size,
            learning_rule=learning_rule,
            stored_patterns=stored_patterns,
            stored_labels=selected_labels,
            coefMatrix=coefMatrix,
            imageGenerator=imageGenerator,
            prediction=prediction,
            binary=binary,
            n_samples=len(stored_patterns),
            color = color,
            save_path=f"internal_confusion_{learning_rule}_{n_train}_binary{binary}_neurons{image_size}.png"
        )

        conf_matrices.append(cm)
        results.append(acc)
        train_labels_list.append(n_train)
        print(f"Internal recall accuracy with {n_train} training images (binary: {binary}, rule: {learning_rule}, neurons: {image_size}): {acc:.2f}")

    return results, conf_matrices

In [109]:
# Initializing the training counts
train_counts = [10, 20 , 30, 40, 50]

In [ ]:

#  Base model
results_hebbian, confs_hebbian = run_hopfield_experiment(
    learning_rule="hebbian",
    hebbian_trainer=hebbian_trainer,
    train_counts= train_counts,
    color = "Blues", # Confusin matrix color
    imageGenerator=imageGenerator,
    prediction=prediction,
    plot_image_reconstruction=plot_image_reconstruction,
    image_size=(32, 32), # Neurons ammount
    binary = True # Black/White or grayscale
)

np.savez("results_hebbian.npz", results=results_hebbian, confs=confs_hebbian)

In [ ]:
# Base model with non-binary
results_non_binary, confs_non_binary = run_hopfield_experiment(
    learning_rule="hebbian",
    hebbian_trainer=hebbian_trainer,
    train_counts= train_counts,
    color=plt.cm.RdPu,
    imageGenerator=imageGenerator,
    prediction=prediction,
    plot_image_reconstruction=plot_image_reconstruction,
    image_size=(64, 64),
    binary= False # Continuous values
)

np.savez("results_non_binary.npz", results=results_non_binary, confs=confs_non_binary, allow_pickle=True)

In [ ]:
# Base model with more neurons
results_more_neurons, confs_more_neurons = run_hopfield_experiment(
    learning_rule="hebbian",
    hebbian_trainer=hebbian_trainer,
    train_counts= train_counts,
    color=plt.cm.RdPu, # Confusion matrix color
    imageGenerator=imageGenerator,
    prediction=prediction,
    plot_image_reconstruction=plot_image_reconstruction,
    image_size=(128, 128), # More neurons
    binary = True # Black/white or grayscale
)

np.savez("results_more_neurons.npz", results=results_more_neurons, confs=confs_more_neurons, allow_pickle=True)

## Plotting learning improvement


In [ ]:
# Loading the saved data.
data_hebbian = np.load("results_hebbian.npz", allow_pickle=True)
data_more_neurons = np.load("results_more_neurons.npz", allow_pickle=True)

In [ ]:
# Extracting the results from the loaded data.
results_hebbian = data_hebbian["results"]
results_more_neurons = data_more_neurons["results"]

In [112]:
# If you do not want to weight 4 hours training (here are the results, uncomment them.)
# results_hebbian = [1.0, 0.45, 0.33, 0.25, 0.16]
# results_more_neurons = [0.9, 0.4, 0.1, 0.15, 0.12]
# train_counts = [10, 20, 30, 40, 50]

In [ ]:
color_hebbian = "#9b5de5"      # violet-purple
color_more_neurons = "#f15bb5" # hot pink

# --- Create figure with subplots ---
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes = axes.flatten()  # make it easy to index as a 1D array

# --- Plot 1: Base model ---
axes[0].plot(train_counts, results_hebbian, 'o-', color=color_hebbian)
axes[0].set_title("Base Hebbian model")
axes[0].set_xlabel("Training images")
axes[0].set_ylabel("Accuracy")
axes[0].set_ylim(0, 1)
axes[0].set_xticks(train_counts)
axes[0].set_yticks(np.arange(0.1, 1.1, 0.1))
axes[0].grid(True)

# --- Plot 2: More neurons model ---
axes[1].plot(train_counts, results_more_neurons, 's-', color=color_more_neurons)
axes[1].set_title("More neurons model")
axes[1].set_xlabel("Training images")
axes[1].set_ylabel("Accuracy")
axes[1].set_ylim(0, 1)
axes[1].set_xticks(train_counts)
axes[1].set_yticks(np.arange(0.1, 1.1, 0.1))
axes[1].grid(True)

# --- Plot 3: Combined comparison ---
axes[2].plot(train_counts, results_hebbian, 'o-', color=color_hebbian, label='Base model')
axes[2].plot(train_counts, results_more_neurons, 's-', color=color_more_neurons, label='More neurons')
axes[2].set_title("All models compared")
axes[2].set_xlabel("Training images")
axes[2].set_ylabel("Accuracy")
axes[2].set_ylim(0, 1)
axes[2].set_xticks(train_counts)
axes[2].set_yticks(np.arange(0.1, 1.1, 0.1))
axes[2].legend()
axes[2].grid(True)

# Hide the unused subplot
axes[3].set_visible(False)

plt.tight_layout()

plt.savefig("hebbian_models_comparison_new.png", dpi=300, bbox_inches='tight')

plt.show()


### Original Code 
#### From: https://github.com/nosratullah/hopfieldNeuralNetwork?tab=readme-ov-file

In [ ]:
"""
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc as sp
import matplotlib.image as img

# import the image and extract
def imageGenerator(imageVector):
    cleanImage = np.zeros([len(imageVector)-1,len(imageVector)-1])
    for i in range(len(imageVector)-1):
        for j in range(len(imageVector)-1):
            if (imageVector[i][j] > 1):
                cleanImage[i][j] = 1
            else:
                cleanImage[i][j] = -1
    noisyImage = cleanImage + np.random.normal(0, 2, [len(image)-1,len(image)-1])

    for i in range(len(image)-1):
        for j in range(len(image)-1):
            if (noisyImage[i][j] >= 0):
                noisyImage[i][j] = 1
            else:
                noisyImage[i][j] = -1

    return cleanImage,noisyImage
# Building up the coefficient matrix
def trainer(vector,oldCoefMat):
    vector = vector.flatten()
    coefMat = np.zeros([len(vector)-1,len(vector)-1])
    if (np.isscalar(oldCoefMat)):
        for i in range(len(vector)-1):
            for j in range(len(vector)-1):
                if (i!=(i-j)):
                    coefMat[i][i-j] = vector[i]*vector[i-j]
    if (np.shape(oldCoefMat) == np.shape(coefMat)):
        for i in range(len(vector)-1):
            for j in range(len(vector)-1):
                if (i!=(i-j)):
                    coefMat[i][i-j] = vector[i]*vector[i-j]
        coefMat = coefMat + oldCoefMat

    vector = np.reshape(vector, [int(np.sqrt(len(vector))),int(np.sqrt(len(vector)))])
    return coefMat

#
def prediction(curuptedVec,coefMat):
    curuptedVec = curuptedVec.flatten()
    predictVec = np.zeros(len(curuptedVec))
    for i in range(len(curuptedVec)-1):
        temp = 0
        for j in range(len(curuptedVec)-1):
             temp += coefMat[i][j] * curuptedVec[j]
        if (temp>0):
            predictVec[i] = 1
        if (temp<0):
            predictVec[i] = -1

    predictVec = np.reshape(predictVec, [int(np.sqrt(len(predictVec))),int(np.sqrt(len(predictVec)))])
    return predictVec


#Import the images
plt.figure(figsize=(15,10))
for i in range(1,4):
    image = img.imread('dataset/pgms/{}.png'.format(i),'w').copy()
    if (i==1):
        vector,noisyVec = imageGenerator(image)
        coefMatrix = trainer(vector,0)
        predictedVec = prediction(noisyVec,coefMatrix)
    else:
        vector,noisyVec = imageGenerator(image)
        coefMatrix = trainer(vector,coefMatrix)
        predictedVec = prediction(noisyVec,coefMatrix)

    plt.subplot(i,4,1)
    plt.imshow(image)
    plt.title('Imported Picture 1')
    plt.subplot(i,4,2)
    plt.imshow(vector);
    plt.title('Cleaned and Squared Picture 1')
    plt.subplot(i,4,3)
    plt.imshow(noisyVec);
    plt.title('Noisy Picture 1')
    plt.subplot(i,4,4)
    plt.imshow(predictedVec);
    plt.title('Recalled Picture 1')

plt.savefig('hopfields.png')
plt.clf()
plt.imshow(coefMatrix)
plt.savefig('matrix.png')
plt.title('Coefficient Matrix')
plt.show()
"""